# プログラムからLLMにアクセスする
既に多くの方は、ChatGPTのような大規模言語モデル（LLM）とやり取りしたことがあるかもしれません。通常、これはUIやアプリケーションを介して行われます。

このNotebookでは、Pythonを使用してLLMに直接アクセスします。
モデルとして、OpenShift AIでServingされた**elyza/ELYZA-japanese-Llama-2-7b-fast-instruct**を利用します。

### 必要なライブラリとインポート

Labの指示に従って適切なワークベンチイメージを選択して起動した場合、必要なすべてのライブラリがすでにインストールされているはずです。もしインストールされていない場合は、次のセルの最初の行のコメントを外して正しいパッケージをすべてインストールしてください。その後、必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

### Langchain

[Langchain](https://www.langchain.com/)は、言語モデルを活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLM（Language Model）に適切にクエリを発行するために手動で書かなければならないすべてのコードを処理します。

まず、LLMインスタンスを作成します。これはLLM APIへのクエリが行われる場所と、モデルに適用されるいくつかのパラメータによって定義されます。たとえば、`max_new_tokens`はモデルが最大512トークン（単語または単語の一部）で回答するよう指示します。`temperature`はここで非常に低く設定されており、モデルに真実に基づいたままであり、あまり「創造的」にならないように指示します。

In [ ]:
# LLM推論APIのURL
inference_server_url = "_INFERENCE_URL_LLM_"

# LLMの定義
llm = VLLMOpenAI(
    openai_api_key="EMPTY", # OpenAI互換のAPIクライアントを使用していますが、モデルはOpenAIではなくOpenShift上で実行されています。そのため、api keyを指定しますが、これは使われません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="/mnt/models/",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

また、モデルに送信するすべてのリクエストに適用する**テンプレート**（プロンプト）も必要です。

モデルにクエリを送信する際、ほとんどの場合、ユーザーが入力した内容をそのまま送ることは望まれません。モデルがそれをどのように扱うかを正確に指示する必要があります。例えば、何をどのように回答するか、回答してはいけないこと、使用するべきトーンなどです。

In [ ]:
system_template_string = """
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。
"""

user_template_string = """
与えられた質問に対して、できるだけ多くの情報を含めて回答して下さい。

### 質問:
{input}

### 回答:
"""

system_template = SystemMessagePromptTemplate.from_template(system_template_string)
user_template = HumanMessagePromptTemplate.from_template(user_template_string)

PROMPT = ChatPromptTemplate.from_messages([system_template, user_template])

Langchainはこれらの要素を簡単に「つなぎ合わせ」、モデルにクエリを送信するために使用する**会話**オブジェクト (conversation) を作成することができます。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

これで、モデルにクエリを送信する準備が整いました。

In [ ]:
query = "人工知能（AI）とは、どのようなものですか？"

conversation.predict(input=query); # 行末の ";" は、最終の出力（ストリームされた回答の繰り返し）を非表示にします。